# Домашнее задание 5
### Выполнила: Кудрявцева Полина Дмитриевна

In [1]:

# импорт библиотек
import pandas as pd
import sqlite3
import math

Прочитать файлы и создать над ними таблицы, где структуры таблиц:

1. articles в виде:

 |-- id: integer (nullable = true)\
 |-- text: string (nullable = true)
 
2. stopwords в виде:

 |-- word: string (nullable = true)

До выполнения задачи изначально обработать данные:
* при парсинге отбросить все символы, которые не являются латинскими буквами;
* привести все слова к нижнему регистру;
* удалить все стоп-слова из articles с помощью таблицы stopwords.

In [2]:
articles = pd.read_csv('data/articles.csv', sep = ';', header = None)
stopwords = pd.read_csv('data/stopwords.csv', sep = ';', header = None)
articles.columns = ['id', 'text']
stopwords.columns = ['word']

In [3]:
# Функция для очистки строки
item = articles.text[0]
def clean_item(item):
    new_item = ''
    for i in item:
        if (i.isalpha() and i.isascii()) or i.isspace():
            new_item+=i.lower()
    new_item = ' '.join([w for w in new_item.split() if w not in list(stopwords['word'])])
    return new_item
print(f'Original: {item}')
print(f'Cleaned: {clean_item(item)}')

Original: Bradley Charles Cooper born January 5 1975 is an American actor and filmmaker He is the recipient of various accolades including a British Academy Film Award and two Grammy Awards in addition to nominations for nine Academy Awards six Golden Globe Awards and a Tony Award Cooper appeared on the Forbes Celebrity 100 three times and on Times list of the 100 most influential people in the world in 2015 His films have grossed $11 billion worldwide and he has placed four times in annual rankings of the worlds highest-paid actors
Cleaned: bradley charles cooper born january american actor filmmaker recipient accolades including british academy film award grammy awards addition nominations academy awards golden globe awards tony award cooper forbes celebrity times times list influential people world films grossed billion worldwide placed times annual rankings worlds highestpaid actors


In [4]:
# Подключимся (и при отсутствии создадим) к базе данных Articles.db
con = sqlite3.connect("./data/Articles.db")

In [5]:
# Инициируем курсор
cur = con.cursor()

In [6]:
# Напишем SQL-запрос для создания таблиц transaction_bd и customer_bd со всеми полями из CSV файлов и выполним его
create_transactuins = '''
CREATE TABLE articles(
    id integer NULL,
    "text" text NULL
);
'''
create_customers = '''
CREATE TABLE stopwords(
    "word" text NULL
);
'''

cur.execute(create_transactuins)
cur.execute(create_customers)
con.commit()

In [7]:
# Очистим текстовое поле из articles перед загрузкой
articles['text'] = articles['text'].apply(clean_item)

In [8]:
# Запишем данные из датафреймов в таблицы
articles.to_sql("articles", con, if_exists="replace", index=False)
stopwords.to_sql("stopwords", con, if_exists="replace", index=False)

733

In [9]:
con.close()

Для каждой комбинации слов  посчитать NPMI и вывести на экран TOP-50 самых популярных коллокаций, отсортированных по убыванию значения NPMI.

In [10]:
# Извлечём все пары слов и отдельные слова, и посчитаем как часто они встречаются
words = dict()
pairs = dict()
for row in articles['text'].str.split():
    for i in range(len(row)):
        if row[i] in words:
            words[row[i]] += 1
        else:
            words[row[i]] = 1
        if i > 0:
            if row[i-1]+' '+row[i] in pairs:
                pairs[row[i-1]+' '+row[i]] += 1
            else:
                pairs[row[i-1]+' '+row[i]] = 1

In [11]:
pairs_df = pd.DataFrame(pairs.items(), columns = ['pair', 'pair_cnt'])
total_pairs = pairs_df.pair_cnt.sum()
pairs_df['pab'] = pairs_df['pair_cnt']/total_pairs

In [12]:
words_df = pd.DataFrame(words.items(), columns = ['word', 'word_cnt'])
#total_words = words_df.word_cnt.sum()
words_df['pw'] = words_df['word_cnt']/total_pairs

In [13]:
pairs_df[['word1','word2']] = pd.DataFrame(pairs_df.pair.str.split().tolist(), index= pairs_df.index)

In [14]:
# 15324 
pairs_df = pairs_df.merge(words_df[['word', 'pw']], how = 'inner', left_on = 'word1', right_on='word')
pairs_df = pairs_df[['pair', 'pab', 'pw' ,'word2']]
pairs_df.columns = ['pair', 'pab', 'pa' ,'word2']

In [15]:
pairs_df = pairs_df.merge(words_df[['word', 'pw']], how = 'inner', left_on = 'word2', right_on='word')
pairs_df = pairs_df[['pair', 'pab', 'pa', 'pw']]
pairs_df.columns = ['pair', 'pab', 'pa' ,'pb']

In [16]:
pairs_df['npmi'] = pairs_df.apply(lambda row: - math.log(row.pab/(row.pa*row.pb))/math.log(row.pab), axis = 1)

Далее у меня есть два варианта прочтения - что же нужно вывести по условию.

А ещё в условии задачи написано, что NPMI находится в диапазоне от -1 до +1. А в примере вывода приведены значения, превышающие 1... Что бы это значило?

In [28]:
# Трактовка А (самые популярные - те, которые встречаются чаще всего, если встречаемость совпадает, стортируем по npmi)
pairs_df.sort_values(by = ['pab', 'npmi'], ascending = False).head(50)[['pair', 'npmi']]


,pair,npmi
7327,united states,0.895088
7809,th century,0.863272
7454,new york,0.879838
3301,induced abortion,0.536487
3329,medical abortion,0.460004
2083,rock roll,0.953913
773,cultural history,0.767466
6365,back sides,0.813356
7450,clipper blade,0.822715
444,award best,0.734608


In [27]:
# Трактовка Б (Отсортировать нужно от максимального nmpi к минимальному, при одинаковых nmpi отдавая предпочтение тем, которые встречаются чаще
pairs_df.sort_values(by = ['npmi', 'pab'], ascending = False).head(50)[['pair', 'npmi']]

,pair,npmi
12436,fish fry,1.0
4428,linings playbook,1.0
5282,nightmare alley,1.0
7895,los angeles,1.0
9875,guardians galaxy,1.0
10625,licorice pizza,1.0
11071,barack obama,1.0
13050,willy wanker,1.0
14620,iberian peninsula,1.0
5695,irina shayk,1.0
